In [140]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

C:\Users\newgh\Anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\Users\newgh\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.o

In [125]:
import pandas as pd
import numpy as np

In [126]:
data = pd.read_csv("ner_dataset.csv", encoding="latin1")
data = data.drop(['POS'], axis =1)
data = data.fillna(method="ffill")

In [138]:
data.head()

,Sentence #,Word,Tag
0,Sentence: 1,Thousands,0
1,Sentence: 1,of,0
2,Sentence: 1,demonstrators,0
3,Sentence: 1,have,0
4,Sentence: 1,marched,0


In [ ]:
data["words"] = data["words"].lemma_.lower()

In [127]:
words = list(set(data['Word'].values))
words.append('PADword')
n_words = len(words)
n_words

35179

In [128]:
binary_tags =[]

In [129]:
for i in range(0,len(data)):
    row=data.iloc[i]
    if row.Tag == 'O':
        binary_tags.append(0)
    else:
        binary_tags.append(1)

In [130]:
data["Tag"]=binary_tags

In [131]:
data.head(12)

,Sentence #,Word,Tag
0,Sentence: 1,Thousands,0
1,Sentence: 1,of,0
2,Sentence: 1,demonstrators,0
3,Sentence: 1,have,0
4,Sentence: 1,marched,0
5,Sentence: 1,through,0
6,Sentence: 1,London,1
7,Sentence: 1,to,0
8,Sentence: 1,protest,0
9,Sentence: 1,the,0


In [132]:
n_tags = 2

In [133]:
def data2sentences(data):
    agg_func = lambda s: [(w, t) for w, t in zip(s["Word"].values.tolist(),s["Tag"].values.tolist())]
    grouped = data.groupby("Sentence #").apply(agg_func)
    sentences = [s for s in grouped]
    return sentences

In [134]:
sentences = data2sentences(data)
print(len(sentences))

47959


In [135]:
max_len = 50
word2idx = {w: i for i, w in enumerate(words)}

In [136]:
import pickle

In [137]:
file = open("myDict.pkl", "wb")
pickle.dump(word2idx, file)
file.close()

In [56]:
from keras.preprocessing.sequence import pad_sequences
X = [[word2idx[w[0]] for w in s] for s in sentences]
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=n_words - 1)

In [57]:
y = [[w[1] for w in s] for s in sentences]
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=0)

In [58]:
from sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1, random_state=2018)

In [59]:
import tensorflow as tf

In [67]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(input_dim=n_words, output_dim=50, input_length=max_len),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=100, return_sequences=True, recurrent_dropout=0)),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1, activation="sigmoid"))
])

In [90]:
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=tf.keras.metrics.Precision())

In [103]:
history = model.fit(X, np.array(y), batch_size=32, epochs=25, verbose=1)

Epoch 1/25
1499/1499 [==============================] - 32s 22ms/step - loss: 0.0039 - precision: 0.9911 0s - loss: 0.0039 - precision
Epoch 2/25
1499/1499 [==============================] - 31s 21ms/step - loss: 0.0035 - precision: 0.9918
Epoch 3/25
1499/1499 [==============================] - 31s 21ms/step - loss: 0.0032 - precision: 0.9925
Epoch 4/25
1499/1499 [==============================] - 31s 21ms/step - loss: 0.0030 - precision: 0.9932
Epoch 5/25
1499/1499 [==============================] - 31s 21ms/step - loss: 0.0027 - precision: 0.9937
Epoch 6/25
1499/1499 [==============================] - 31s 21ms/step - loss: 0.0024 - precision: 0.9943 1s - loss: 0.0024 -
Epoch 7/25
1499/1499 [==============================] - 32s 21ms/step - loss: 0.0023 - precision: 0.9947
Epoch 8/25
1499/1499 [==============================] - 32s 21ms/step - loss: 0.0021 - precision: 0.9949
Epoch 9/25
1499/1499 [==============================] - 33s 22ms/step - loss: 0.0019 - precision: 0.9951
Epoch

In [143]:
np.array([X_te[i]]).shape

(1, 50)

In [142]:
i = 777
p = model.predict(np.array([X_te[i]]))

In [120]:
for w, pred in zip(X_te[i], p[0]):
    print("{:10}: {}".format(words[w], pred))

Negotiators: [5.0730055e-08]
in        : [8.067875e-07]
Beijing   : [1.]
will      : [1.0808631e-09]
resume    : [8.107382e-08]
work      : [5.328959e-06]
Sunday    : [1.]
on        : [6.8496276e-15]
forging   : [5.18145e-10]
an        : [5.170918e-13]
agreement : [2.8706592e-13]
to        : [1.7745342e-11]
end       : [1.4305899e-05]
the       : [0.00013121]
North     : [0.99999356]
Korean    : [1.]
nuclear   : [6.769076e-08]
crisis    : [3.6449947e-07]
.         : [1.2244275e-10]
PADword   : [1.5352912e-12]
PADword   : [1.2419983e-13]
PADword   : [8.943167e-14]
PADword   : [4.3314294e-13]
PADword   : [1.0077966e-12]
PADword   : [1.2528053e-12]
PADword   : [1.3306721e-12]
PADword   : [1.3775681e-12]
PADword   : [1.5221846e-12]
PADword   : [1.8184178e-12]
PADword   : [2.2739655e-12]
PADword   : [2.907758e-12]
PADword   : [3.785075e-12]
PADword   : [5.009907e-12]
PADword   : [6.7008647e-12]
PADword   : [8.942819e-12]
PADword   : [1.1697588e-11]
PADword   : [1.4692421e-11]
PADword   : [1

In [123]:
model.save('./namedER/model')

INFO:tensorflow:Assets written to: ./namedER/model\assets


INFO:tensorflow:Assets written to: ./namedER/model\assets
